# 사전 설정

In [2]:
from pykrx import stock
import pandas as pd
import build_universes

# 유동성 필터링 ETF 목록 로드

In [3]:
etf_pool = build_universes.get_liquid_etf_pool("20231228")
print(etf_pool)


[20231228 기준 데이터 수집 및 1차 유동성 필터링 (최종 결정판)]
  -> 총 723개 ETF의 유효성을 검증하고 정보를 결합합니다...


ETF 정보 유효성 검증: 100%|██████████| 723/723 [00:00<00:00, 90403.11it/s]


  -> 총 723개 ETF 중 163개가 유효성 검증 및 최소 거래대금 기준을 통과했습니다.
     ticker                   name         거래대금
7    361580             RISE 200TR   2563185070
13   148020               RISE 200   8732885245
14   465330         RISE 2차전지TOP10   2572813945
15   465350  RISE 2차전지TOP10인버스(합성)  21452270870
16   422420           RISE 2차전지액티브  11013790770
..      ...                    ...          ...
668  152100               PLUS 200   2130732285
673  453010            PLUS KOFR금리   1179239310
677  449450               PLUS K방산   2035674540
682  161510              PLUS 고배당주   2731840291
719  470310   UNICORN 생성형AI강소기업액티브  12043630835

[163 rows x 3 columns]


## ETF 목록 저장

In [5]:
etf_pool_temp = etf_pool.copy()
etf_pool_temp['factor'] = etf_pool_temp['name'].apply(build_universes.classify_etf_engine)
etf_pool_temp.to_excel(f"liquid_etfs_pool_20231228.xlsx", index=False)

# 안전 투자형 전략

## 

2024년 백테스트를 위한 투자 유니버스 구성
1. 데이터 수집 기준일 (Look-ahead Bias 방지): 20231228

2. KRX에 상장된 모든 ETF 정보 수집 중...


TypeError: got an unexpected keyword argument 'ticker'